In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    groq_api_key="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"
)

response=llm.invoke("The first person to land on the moon was ...")
print(response.content)

Neil Armstrong. He stepped onto the moon's surface on July 20, 1969, during the Apollo 11 mission.


In [8]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.foundit.in/job/senior-program-manager-microsoft-india-31247665?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic")
page_data = loader.load().pop().page_content
print(page_data)



















Senior Program Manager Job for 0 - 2 Year of Exp In Microsoft India - 31247665 | foundit india 
JobsServicesBoostFeatured profileProfile highlighterCareer BoosterBest SellerResume WritingResume builderLinkedin MakeoverPrepMock interviewInterview PreparationLearnSkill AssessmentDegree ProgramsCareer AdviceCertificationsNew Search JobsPlease enter a job title/role/skill to searchPlease enter a job title/role/skill to searchPlease enter a job title/role/skill to searchFresher (< 1 year)Recent SearchesSearchSearch by job, company or skillsLoginRegisterSenior Program ManagerMicrosoftHighlightsJob DescriptionMore InfoRecruiter InfoIndia0-2 Years2 months ago83 Viewed1 AppliedJob DescriptionMicrosoft is a company where passionate innovators come to collaborate, envision what can be and take their careers to levels they cannot achieve anywhere else. This is a world of more possibilities, more innovation, more openness, and the sky is the limit thinking a cloud-enabled world.Mi

In [10]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
# print(res.content)
type(res.content)

[
  {
    "role": "Senior Program Manager",
    "experience": "0-2 years",
    "skills": "technical program manager, Marketing, operations team, Development, Commerce, Engineering, operations, complex, technical expertise, successful delivery, challenging environments, erp applications, product direction, Hr, Design, crm, Customer Support, Sales, Finance, Services, Service, automation",
    "description": "Microsoft is a company where passionate innovators come to collaborate, envision what can be and take their careers to levels they cannot achieve anywhere else. This is a world of more possibilities, more innovation, more openness, and the sky is the limit thinking a cloud-enabled world.Microsoft Dynamics 365 and Microsoft Power Platform are at the heart of our customers digital transformation journey. Our products meet today's challenges by bringing customers and business together with the next generation of CRM and ERP applications. Recognised as a platform leader across Sales, Mar

str

In [11]:
# Change the output in json
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Senior Program Manager',
  'experience': '0-2 years',
  'skills': 'technical program manager, Marketing, operations team, Development, Commerce, Engineering, operations, complex, technical expertise, successful delivery, challenging environments, erp applications, product direction, Hr, Design, crm, Customer Support, Sales, Finance, Services, Service, automation',
  'description': "Microsoft is a company where passionate innovators come to collaborate, envision what can be and take their careers to levels they cannot achieve anywhere else. This is a world of more possibilities, more innovation, more openness, and the sky is the limit thinking a cloud-enabled world.Microsoft Dynamics 365 and Microsoft Power Platform are at the heart of our customers digital transformation journey. Our products meet today's challenges by bringing customers and business together with the next generation of CRM and ERP applications. Recognised as a platform leader across Sales, Marketing, Servic

In [13]:
import pandas as pd

df=pd.read_csv("my_portfolio.csv")
df.head()

,Techstack,Links
0,Excel,https://github.com/aditya345-coder/Excel-Proje...
1,"Java, AVT, Swing",https://github.com/aditya345-coder/Game-dev
2,Python,https://github.com/aditya345-coder/Python-Usef...
3,"Python, Django, Machine Learning, Deep Learnin...",https://github.com/aditya345-coder/Social-Medi...
4,"Python, Flask, Machine Learning, Deep Learning...",https://github.com/aditya345-coder/Wine-Qualit...


In [14]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [17]:
# links = collection.query(query_texts=["Experience in Python", "Expertise in Django"], n_results=2).get('metadatas', [])
# links

links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links


TypeError: list indices must be integers or slices, not str

In [18]:
job=json_res
job["Skills"]

TypeError: list indices must be integers or slices, not str

In [19]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Aditya, a BTech computer science student at DIT University. I have experience in python, flask, django, html, css,
        html, css, deep learning, machine learning, computer vision, NLP.
        I have done 2 internship one in IIIT Dharwad where I annoted the data using CVAT and then created model for the correct unpaved road 
        prediction with my colleague and second in a company called kohli LLP where we worked on time sereis data and trained the model using different 
        time series models such as TFT, neuralprophet and hybird models etc.
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase my portfolio: {link_list}
        Remember you are Aditya. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expertise in Technical Program Management and Development for Microsoft Dynamics 365

Dear Hiring Manager,

I came across the job description for a Senior Program Manager at Microsoft, and I am excited to express my interest in the role. As a BTech computer science student at DIT University, I have developed a strong foundation in technical program management and development, which aligns with the requirements of the position.

With experience in technologies such as Python, Flask, Django, HTML, CSS, deep learning, machine learning, computer vision, and NLP, I am confident in my ability to manage complex technical programs and deliver successful outcomes. My experience in working with various technologies has equipped me with the skills to analyze problems, identify solutions, and implement them effectively.

In my previous internships, I have worked on projects that involved data annotation using CVAT, creating models for unpaved road prediction, and working with time series 